In [1]:
import os
import pandas as pd
from collections import Counter
import csv

In [245]:
gradebook = pd.read_csv('./gradebook_PMBC148_grades.csv')
## replace NAs with zeroes
gradebook = gradebook.fillna(0)
# gradebook = gradebook[gradebook['Final Exam Percent']!=0].reset_index(drop=True)
gradebook = gradebook.drop(columns=['Unnamed: 0'])
gradebook

,First Name,Last Name,SID,Email,section_name,Problem Set 1,Problem Set 1 - Max Points,Problem Set 1 - Submission Time,Problem Set 1 - Lateness (H:M:S),Problem Set 2,...,Problem Set 7 Percent,Problem Set 8 Percent,Problem Set 9 Percent,Problem Sets Final Grade,Exam 1 Percent,Exam 2 Percent,Exam 3 Percent,Exam 4 Percent,Final Exam Percent,Final Grade
0,Aaron,Kirby,3033452840,aerk@berkeley.edu,mcellbi-c148-dis-102-in-person,0.0,42.5,0,00:00:00,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Ada,Barreto,3033585052,adabarreto@berkeley.edu,plantbi-c148-dis-101-in-person,31.8,42.5,2022-01-25 04:01:08 -0800,00:00:00,34.0,...,0.441667,0.553571,0.135593,0.552803,0.781250,0.844444,0.814286,0.540816,0.685484,0.721331
2,Adrien,Stroumza,3034644016,a.stroumza@berkeley.edu,mcellbi-c148-dis-102-in-person,37.0,42.5,2022-01-25 07:57:40 -0800,00:00:00,34.5,...,0.800000,0.901786,0.601695,0.690673,0.781250,0.688889,0.619048,0.948980,0.862903,0.789485
3,Albert,Duenas,3036941765,albertduenas@berkeley.edu,mcellbi-c148-dis-102-in-person,29.0,42.5,2022-01-25 00:35:44 -0800,00:00:00,33.5,...,0.000000,0.000000,0.000000,0.237705,0.593750,0.288889,0.480952,0.581633,0.483871,0.442179
4,Albert,Tran,3034573283,albert.tran.100@berkeley.edu,mcellbi-c148-dis-102-in-person,38.5,42.5,2022-01-24 15:05:08 -0800,00:00:00,36.5,...,1.000000,1.000000,1.000000,0.991503,0.640625,0.733333,0.976190,0.897959,0.887097,0.893815
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,Xue Ying,Zhu,3034885231,xueyingzhu@berkeley.edu,plantbi-c148-dis-101-in-person,0.0,42.5,0,00:00:00,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
86,Xueyang,Jiang,3035902921,xyjiang_stephen03@berkeley.edu,plantbi-c148-dis-102-in-person,37.0,42.5,2022-01-23 19:21:22 -0800,00:00:00,36.5,...,0.000000,0.000000,0.000000,0.261588,0.000000,0.000000,0.000000,0.000000,0.000000,0.036622
87,Yipei,Tang,3036127223,yit001@berkeley.edu,mcellbi-c148-dis-102-in-person,31.5,42.5,2022-01-27 09:06:45 -0800,49:06:45,34.5,...,0.908333,0.982143,0.906780,0.928465,0.859375,0.911111,0.861905,0.877551,0.903226,0.897655
88,Zenna,Lee,3036060468,lsh18@berkeley.edu,mcellbi-c148-dis-102-in-person,41.5,42.5,2022-01-25 02:54:14 -0800,00:00:00,38.0,...,0.983333,1.000000,0.991525,0.991261,0.765625,0.933333,0.861905,0.979592,0.911290,0.917703


In [247]:
len(gradebook['SID'].unique())

90

In [248]:
import glob
from subprocess import call
## parse quiz grades, exported from bcourses

quiz_list = os.listdir('./quizzes')

for q in glob.glob('quizzes/*'):
    
    new_q="".join(q.split(' '))
    old_q="\ ".join(q.split(' '))
    new_q2='Quiz'.join(new_q.split("QuizQuiz"))
    cmd='mv {} {}'.format(old_q,new_q2)
#     print(cmd)
#     call(cmd,shell=True)

In [249]:
total_quiz_count=[]

for q in glob.glob('quizzes/*'):
    total_quiz_count.append(q)
#     print(q)
total_quiz_count=sorted(total_quiz_count)
print(len(total_quiz_count))
# print(total_quiz_count)

23


In [250]:
attempts_all_students = []

quiz_scores = []

quiz_max_total = 40 ## 23 quizzes times 2 minus 6 drops

student2attemps={}

for sid in gradebook['SID']:
    attempts_per_student = []

    for quiz in total_quiz_count:
        df = pd.read_csv(quiz)
        counter = Counter(df['sis_id'].tolist())
        attempts = 0
        ## basically just counting to see how many times each SID shows up in each quiz
        for key in counter:
            if str(sid) in str(key):
                attempts = counter[key]
        if attempts >= 2:
            attempts_per_student.append(2)
        else:
            attempts_per_student.append(attempts)
    attempts_all_students.append(attempts_per_student)
    student2attemps[sid]=attempts_per_student
    
    if sid == 3035932977: # drop first 10 quizzes for Ken Chen 3035932977
        attempts_per_student = attempts_per_student[10:]
        quiz_score_per_student = sum(attempts_per_student)/26
    elif sid == 3036743721: # drop an extra quiz for Owen Ong 3036743721
        quiz_score_per_student = sum(attempts_per_student)/39
    else:
        quiz_score_per_student = sum(attempts_per_student)/quiz_max_total
    quiz_scores.append(min([quiz_score_per_student, 1.0]))

In [251]:
len(student2attemps)

90

In [252]:
gradebook['Quiz Attempts'] = attempts_all_students
gradebook['Quiz Grade'] = quiz_scores
gradebook[['First Name','Last Name','Quiz Attempts','Quiz Grade',]]

,First Name,Last Name,Quiz Attempts,Quiz Grade
0,Aaron,Kirby,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.00
1,Ada,Barreto,"[2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",1.00
2,Adrien,Stroumza,"[2, 2, 2, 1, 1, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",1.00
3,Albert,Duenas,"[2, 2, 2, 2, 2, 0, 1, 1, 2, 1, 1, 1, 1, 1, 1, ...",0.60
4,Albert,Tran,"[2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",1.00
...,...,...,...,...
85,Xue Ying,Zhu,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.00
86,Xueyang,Jiang,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.00
87,Yipei,Tang,"[2, 2, 1, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, ...",0.95
88,Zenna,Lee,"[2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",1.00


In [253]:
## i clicker
iclicker = pd.read_csv('./iClicker_GradesExport_Reef_05-05-22.csv', index_col=1)
iclicker = iclicker[iclicker['Name'].notna()]

iclicker_percent = []
iclicker_final_score = []

name2iclicker={}
for email in gradebook['Email']:
    name="_".join([gradebook.set_index('Email').loc[email,'First Name'],\
                   gradebook.set_index('Email').loc[email,'Last Name']])
    try:
        iclicker_score = iclicker.loc[email]['Total %']
    except KeyError:
        iclicker_score = '0%'
    iclicker_score = float(iclicker_score[:-1])/100
    iclicker_percent.append(iclicker_score)
    if email == 'ken_chen@berkeley.edu': # drop clickers for the first few weeks
        if iclicker_score >= 0.46:
            iclicker_final_score.append(1)
        else:
            iclicker_final_score.append(0)
    elif email == 'heliasamani@berkeley.edu':
        if iclicker_score >= 0.5:
            iclicker_final_score.append(1)
    elif iclicker_score >= 0.7:
        iclicker_final_score.append(1)
    else:
        iclicker_final_score.append(0)
        
    name2iclicker[name]=iclicker_percent
    
iclicker

,Name,Student ID,Total,Total %,Performance,Participation,Session 31 - 04/28/22,Session 30 - 04/21/22,Session 29 - 04/19/22,Session 28 - 04/14/22,...,Session 12 - 02/24/22,Session 11 - 02/22/22,Session 10 - 02/17/22,Session 8 - 02/10/22,Session 7 - 02/03/22,Session 6 - 02/01/22,Session 5 - 01/27/22,Session 4 - 01/25/22,Session 3 - 01/20/22,Session 2 - 01/18/22
Email,,,,,,,,,,,,,,,,,,,,,
andreabarca@berkeley.edu,"Abarca, Andrea",andreabarca,21.0,91.3%,0.0,21.0,1.0,1.0,1.0,1.0,...,No Response,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0
sydneyabelson@berkeley.edu,"Abelson, Sydney",1671294,22.0,95.65%,0.0,22.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
benjaminayers@berkeley.edu,"Ayers, Benjamin",3033532272,18.0,78.26%,0.0,18.0,1.0,1.0,No Response,1.0,...,1.0,1.0,1.0,1.0,No Response,1.0,1.0,1.0,1.0,1.0
adabarreto@berkeley.edu,"Barreto, Ada",1610959,17.0,73.91%,0.0,17.0,1.0,1.0,1.0,1.0,...,1.0,1.0,No Response,1.0,0.0,1.0,No Response,No Response,1.0,1.0
danielaberman@berkeley.edu,"Berman, Daniela",danielaberman,23.0,100.0%,0.0,23.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
spencer_zezulka@berkeley.edu,"Zezulka, Spencer",3034200550,4.0,17.39%,0.0,4.0,No Response,No Response,No Response,No Response,...,No Response,No Response,No Response,No Response,1.0,No Response,No Response,1.0,1.0,1.0
zhanglanxin125@berkeley.edu,"Zhang, Lanxin",zhanglanxin125,20.0,86.96%,0.0,20.0,1.0,1.0,1.0,1.0,...,1.0,1.0,No Response,1.0,1.0,1.0,No Response,0.0,1.0,1.0
weilinzhang@berkeley.edu,"Zhang, Weilin",1797438,12.0,52.17%,0.0,12.0,No Response,No Response,No Response,No Response,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [254]:
gradebook['iClicker Percent'] = iclicker_percent
gradebook['iClicker Final Score'] = iclicker_final_score
gradebook[['First Name','Last Name','Email',\
           'iClicker Percent','iClicker Final Score',]]

,First Name,Last Name,Email,iClicker Percent,iClicker Final Score
0,Aaron,Kirby,aerk@berkeley.edu,0.0870,0
1,Ada,Barreto,adabarreto@berkeley.edu,0.7391,1
2,Adrien,Stroumza,a.stroumza@berkeley.edu,0.6522,0
3,Albert,Duenas,albertduenas@berkeley.edu,0.3043,0
4,Albert,Tran,albert.tran.100@berkeley.edu,1.0000,1
...,...,...,...,...,...
85,Xue Ying,Zhu,xueyingzhu@berkeley.edu,0.0000,0
86,Xueyang,Jiang,xyjiang_stephen03@berkeley.edu,0.1739,0
87,Yipei,Tang,yit001@berkeley.edu,0.7826,1
88,Zenna,Lee,lsh18@berkeley.edu,0.3043,0


In [255]:
gradebook[gradebook['iClicker Percent']<0.7].\
sort_values('iClicker Percent',ascending=False)[['First Name','Last Name','Email',\
           'iClicker Percent','iClicker Final Score',]]

,First Name,Last Name,Email,iClicker Percent,iClicker Final Score
10,Angela,Zablocki,angelazablocki@berkeley.edu,0.6957,0
2,Adrien,Stroumza,a.stroumza@berkeley.edu,0.6522,0
58,Madeline,Foster,cgdftmrfc@berkeley.edu,0.6087,0
64,Matthew,Cazas,m.cazas492@berkeley.edu,0.6087,0
71,Priya,Kallu,hkallu@berkeley.edu,0.5652,0
47,Ken,Chen,ken_chen@berkeley.edu,0.5652,1
83,Weilin,Zhang,weilinzhang@berkeley.edu,0.5217,0
72,Rachel,Ong,rachelong1550@berkeley.edu,0.5217,0
63,Mateo,Ortiz Ambrosio,mateo.ortizambrosio@berkeley.edu,0.5217,0
34,Helia,Samani,heliasamani@berkeley.edu,0.5217,1


In [235]:
# iclicker2 = iclicker.filter(regex='Session')

# for i,row in iclicker2.iterrows():
#     for c in iclicker2:
#         if iclicker2.loc[i,c] == 'No Response':
#             iclicker2.loc[i,c]=0
#         elif iclicker2.loc[i,c] == '1.0':
#             iclicker2.loc[i,c]=1
#         elif iclicker2.loc[i,c] == '0.0':
#             iclicker2.loc[i,c]=0
#         else:
#             iclicker2.loc[i,c]=int(iclicker2.loc[i,c])

# iclicker2.insert(0,'Session_sum',iclicker2.sum(axis=1))

# iclicker2 = iclicker2.reset_index()
# iclicker2 = iclicker2[iclicker2['Email'].notna()]
# iclicker2

# iclicker3=iclicker.reset_index()[['Email',\
#                                   'Name']].merge(iclicker2,on='Email',how='right').set_index('Email')
# iclicker3

In [256]:
## calculate problem set score with drops

## calculate percentages
gradebook['Problem Set 1 Percent'] = gradebook['Problem Set 1'] / gradebook['Problem Set 1 - Max Points']
gradebook['Problem Set 2 Percent'] = gradebook['Problem Set 2'] / gradebook['Problem Set 2 - Max Points']
gradebook['Problem Set 3 Percent'] = gradebook['Problem Set 3'] / gradebook['Problem Set 3 - Max Points']
gradebook['Problem Set 4 Percent'] = gradebook['Problem Set 4'] / gradebook['Problem Set 4 - Max Points']
gradebook['Problem Set 5 Percent'] = gradebook['Problem Set 5'] / gradebook['Problem Set 5 - Max Points']
gradebook['Problem Set 6 Percent'] = gradebook['Problem Set 6'] / gradebook['Problem Set 6 - Max Points']
gradebook['Problem Set 7 Percent'] = gradebook['Problem Set 7'] / gradebook['Problem Set 7 - Max Points']
gradebook['Problem Set 8 Percent'] = gradebook['Problem Set 8'] / gradebook['Problem Set 8 - Max Points']
gradebook['Problem Set 9 Percent'] = gradebook['Problem Set 9'] / gradebook['Problem Set 9 - Max Points']
problem_set_final_grade = []

for index, row in gradebook.iterrows():
    all_ps_scores = [
        row['Problem Set 1 Percent'],
        row['Problem Set 2 Percent'],
        row['Problem Set 3 Percent'],
        row['Problem Set 4 Percent'],
        row['Problem Set 5 Percent'],
        row['Problem Set 6 Percent'],
        row['Problem Set 7 Percent'],
        row['Problem Set 8 Percent'],
        row['Problem Set 9 Percent']
    ]
    if row['First Name'] == 'Ken' and row['Last Name'] == 'Chen': # drop ps 1, 2 and 3
        del all_ps_scores[0:3]
        all_ps_scores = [score * (1/6) for score in all_ps_scores]
    else: # drop 2 lowest PS
        all_ps_scores.remove(min(all_ps_scores))
        all_ps_scores.remove(min(all_ps_scores))
        all_ps_scores = [score * (1/7) for score in all_ps_scores]
    problem_set_final_grade.append(sum(all_ps_scores))
gradebook['Problem Sets Final Grade'] = problem_set_final_grade

In [257]:
# calculate exam percentages
gradebook['Exam 1 Percent'] = gradebook['Exam 1'] / gradebook['Exam 1 - Max Points']
gradebook['Exam 2 Percent'] = gradebook['Exam 2'] / gradebook['Exam 2 - Max Points']
gradebook['Exam 3 Percent'] = gradebook['Exam 3'] / gradebook['Exam 3 - Max Points']
gradebook['Exam 4 Percent'] = gradebook['Exam 4'] / gradebook['Exam 4 - Max Points']
gradebook['Final Exam Percent'] = gradebook['Final Exam'] / gradebook['Final Exam - Max Points']

In [258]:
## now calculate final grade with scaling factors

In [259]:
ps_scaling_factor = 0.14
exam_1_scaling_factor = 0.11
exam_2_scaling_factor = 0.17
exam_3_scaling_factor = 0.17
exam_4_scaling_factor = 0.2
final_scaling_factor = 0.28
scaling_factors = [ps_scaling_factor, exam_1_scaling_factor,
                  exam_2_scaling_factor, exam_3_scaling_factor, 
                  exam_4_scaling_factor, final_scaling_factor]

In [260]:
final_grades = []

for index, row in gradebook.iterrows():
    final_percentages = [
        row['Problem Sets Final Grade'],
        row['Exam 1 Percent'],
        row['Exam 2 Percent'],
        row['Exam 3 Percent'],
        row['Exam 4 Percent'],
        row['Final Exam Percent']
    ]
    # get index of lowest score
    min_index=final_percentages.index(min(final_percentages))
    individual_scaling_factors = scaling_factors.copy()
    individual_scaling_factors[min_index] = round(individual_scaling_factors[min_index]-0.1,2)
    final_percentages.append(row['Quiz Grade'])
    final_percentages.append(row['iClicker Final Score'])
    individual_scaling_factors.append(0.015)
    individual_scaling_factors.append(0.015)
    # forced drops for some students
    if row['First Name'] == 'Crystal' and row['Last Name'] == 'Le': # exam 4
        individual_scaling_factors = [0.14, 0.11, 0.17, 0.17, 0.1, 0.28, 0.015, 0.015]
    elif row['First Name'] == 'Weilin' and row['Last Name'] == 'Zhang': # exam 4
        individual_scaling_factors = [0.14, 0.11, 0.17, 0.17, 0.1, 0.28, 0.015, 0.015]
    elif row['First Name'] == 'Albert' and row['Last Name'] == 'Duenas': # exam 3
        individual_scaling_factors = [0.14, 0.11, 0.17, 0.07, 0.2, 0.28, 0.015, 0.015]
    elif row['First Name'] == 'Ken' and row['Last Name'] == 'Chen': # full drop exam 1
        individual_scaling_factors = [0.14, 0, 0.17, 0.17, 0.2, 0.28, 0.015, 0.015]
    ## multiply the two lists
    scaled_final_percentages = [a*b for a,b in zip(final_percentages,individual_scaling_factors)]
    final_grade = sum(scaled_final_percentages)
    if row['First Name'] == 'Ken' and row['Last Name'] == 'Chen': # to deal with missing 1%
        final_grade = final_grade * (100/99)
    final_grades.append(final_grade)

In [261]:
gradebook['Final Grade'] = final_grades

In [262]:
gradebook.to_csv('gradebook_PMBC148_grades_ClareV2.csv')

In [ ]:
## data exploration

In [263]:
gradebook[(gradebook['Final Grade'] < 0.6) & (gradebook['Final Grade'] > 0.1)]

,First Name,Last Name,SID,Email,section_name,Problem Set 1,Problem Set 1 - Max Points,Problem Set 1 - Submission Time,Problem Set 1 - Lateness (H:M:S),Problem Set 2,...,Problem Set 7 Percent,Problem Set 8 Percent,Problem Set 9 Percent,Problem Sets Final Grade,Exam 1 Percent,Exam 2 Percent,Exam 3 Percent,Exam 4 Percent,Final Exam Percent,Final Grade
3,Albert,Duenas,3036941765,albertduenas@berkeley.edu,mcellbi-c148-dis-102-in-person,29.0,42.5,2022-01-25 00:35:44 -0800,00:00:00,33.5,...,0.000000,0.000000,0.000000,0.237705,0.593750,0.288889,0.480952,0.581633,0.483871,0.442179
8,Andrea,Abarca Cevallos,3034999956,andreabarca@berkeley.edu,mcellbi-c148-dis-102-in-person,33.1,42.5,2022-01-25 04:21:14 -0800,00:00:00,29.5,...,0.766667,0.866071,0.618644,0.712357,0.390625,0.177778,0.433333,0.489796,0.459677,0.480979
19,Chris,Dong,3036177104,cxdong@berkeley.edu,mcellbi-c148-dis-102-in-person,33.0,42.5,2022-01-24 20:31:36 -0800,00:00:00,34.5,...,0.866667,0.982143,0.932203,0.918915,0.937500,0.955556,0.933333,0.000000,0.000000,0.555134
22,Crystal,Le,3035688694,crystle@berkeley.edu,plantbi-c148-dis-101-in-person,38.0,42.5,2022-01-25 02:35:51 -0800,00:00:00,35.5,...,0.966667,1.000000,0.991525,0.954561,0.390625,0.466667,0.516667,0.632653,0.520161,0.582684


In [267]:
Pierre_gradebook= pd.read_csv('./gradebook_PMBC148_grades.csv')[['First Name','Last Name','Final Grade']]
Clare_gradebook= pd.read_csv('./gradebook_PMBC148_grades_ClareV2.csv')[['First Name','Last Name','Final Grade']]

In [272]:
Mdb=Pierre_gradebook.rename(columns={'Final Grade':'Final Grade by Pierre'}).\
merge(Clare_gradebook.rename(columns={'Final Grade':'Final Grade by Clare'}),\
      on=['First Name','Last Name'],how='left')
Mdb[Mdb['Final Grade by Pierre']!=Mdb['Final Grade by Clare']]

,First Name,Last Name,Final Grade by Pierre,Final Grade by Clare
34,Helia,Samani,0.872856,0.887856
47,Ken,Chen,0.949696,0.947947
